In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import os
import tqdm
import scipy
from scipy import signal
from tqdm import tnrange
import seaborn as sns
from scipy.stats import norm,entropy,linregress
from scipy.optimize import minimize, curve_fit
from scipy.io import savemat
import multiprocess as mp
from multiprocess import Pool
import sys
import warnings
warnings.filterwarnings('ignore')
sns.set_context('poster', font_scale=1.1)
cmap = plt.rcParams['axes.prop_cycle'].by_key()['color']
modulename = 'multiprocess'

mpl.rcParams['svg.fonttype'] = 'none'
sns.set_context('poster', font_scale=1.1)

In [2]:
def calc_plam(num_pulses,lam1,lam2):
    pow_lam1 = np.power(lam1,num_pulses)
    pow_lam2 = np.power(lam2,num_pulses)
    exp_lam1 = np.exp(-lam1)
    exp_lam2 = np.exp(-lam2)
    return pow_lam1*exp_lam1/(pow_lam1*exp_lam1+pow_lam2*exp_lam2)

def compute_pcorrect(choices,Nps,Nb,sig_dN,sig_Np,Nmin=1):
    sig_Nps = sig_Np[Nps-Nmin]
    sig = np.sqrt(sig_Nps**2+sig_dN**2)
    phigh = norm.cdf((Nps-Nb)/sig)
    pcorrect = (1-choices)+(choices*2-1)*phigh
    return pcorrect

def compute_pcorrect_weighted(choices,weighted_odor,Nps,Nb,sig_dN,sig_Np,Nmin=1):
    sig_Nps = sig_Np[Nps-Nmin]
    sig = np.sqrt(sig_Nps**2+sig_dN**2)
    phigh = norm.cdf((weighted_odor-Nb)/sig)
    pcorrect = (1-choices)+(choices*2-1)*phigh
    return pcorrect

def compute_phigh(Np,Nb,sig_dN,sig_Np):
    sig = np.sqrt(sig_Np**2+sig_dN**2)
    phigh = norm.cdf((Np-Nb)/sig)
    return phigh

def compute_phigh_sq(Np,Nb,sig_dN,ksig,Nmin=1,Nmax=18):
    sig_Np = np.sqrt(ksig*(np.arange(Nmin,Nmax+1)))
    sig = np.sqrt(sig_Np**2+sig_dN**2)
    phigh = norm.cdf((Np-Nb)/sig)
    return phigh

def compute_phigh_lin(Np,Nb,sig_dN,ksig,sig_0,Nmin=1,Nmax=18):
    sig_Np = ksig*(np.arange(Nmin,Nmax+1)) + sig_0
    sig = np.sqrt(sig_Np**2+sig_dN**2)
    phigh = norm.cdf((Np-Nb)/sig)
    return phigh

def compute_phigh_lin_sub(Np,Nb,sig_dN,ksig,sig_0,Nmin=1,Nmax=18): #ksig,sig_0,sub
    #sig_Np = ksig*(np.arange(Nmin,Nmax+1)) + sig_0
    sub_Np = np.zeros(4) #sig_0*np.ones(4)
    sig_Np = np.concatenate([sub_Np,ksig*(np.arange(5,Nmax+1,1))+sig_0]) # + sig_0
    print(sig_Np)
    sig = np.sqrt(sig_Np**2+sig_dN**2)   #Why this is not multiplied by 2
    phigh = norm.cdf((Np-Nb)/sig)
    return phigh

def compute_phigh_pow(Np,Nb,sig_dN,ksig,ind,Nmin=1,Nmax=18):
    sig_Np = ksig*np.power((np.arange(Nmin,Nmax+1)),ind)
    sig = np.sqrt(sig_Np**2+sig_dN**2)
    phigh = norm.cdf((Np-Nb)/sig)
    return phigh

def compute_phigh_const(Np,Nb,sig_dN,Nmin=1,Nmax=18):
    sig = sig_dN
    phigh = norm.cdf((Np-Nb)/sig)
    return phigh

def compute_logp_correct(choices,Nps,Nb,sig_dN,sig_Np,Nmin=1):
    pcorrect = compute_pcorrect(choices,Nps,Nb,sig_dN,sig_Np,Nmin=1)
    log_pcorrect = np.log(pcorrect)
    return log_pcorrect.sum()

def compute_logp_correct_weighted(choices,weighted_odor,Nps,Nb,sig_dN,sig_Np):
    pcorrect = compute_pcorrect_weighted(choices,weighted_odor,Nps,Nb,sig_dN,sig_Np)
    log_pcorrect = np.log(pcorrect)
    return log_pcorrect.sum()

def compute_logp_correct_sq(choices,Nps,Nb,sig_dN,ksig,Nmin=1,Nmax=18):
    sig_Np = np.sqrt(ksig*(np.arange(Nmin,Nmax+1)))
    return compute_logp_correct(choices,Nps,Nb,sig_dN,sig_Np)

def compute_logp_correct_sq_weighted(choices,weighted_odor,Nps,Nb,sig_dN,ksig,Nmin=1,Nmax=18):
    sig_Np = np.sqrt(ksig*(np.arange(Nmin,Nmax+1)))
    return compute_logp_correct_weighted(choices,weighted_odor,Nps,Nb,sig_dN,sig_Np)
    
def compute_logp_correct_lin(choices,Nps,Nb,sig_dN,ksig,sig_0,Nmin=1,Nmax=18):
    sig_Np = ksig*(np.arange(Nmin,Nmax+1)) + sig_0
    pcorrect = compute_pcorrect(choices,Nps,Nb,sig_dN,sig_Np,Nmin)
    log_pcorrect = np.log(pcorrect)
    return log_pcorrect.sum()

def compute_logp_correct_lin_weighted(choices,weighted_odor,Nps,Nb,sig_dN,ksig,sig_0,Nmin=1,Nmax=18):
    sig_Np = ksig*(np.arange(Nmin,Nmax+1)) + sig_0
    pcorrect = compute_pcorrect_weighted(choices,weighted_odor,Nps,Nb,sig_dN,sig_Np)
    log_pcorrect = np.log(pcorrect)
    return log_pcorrect.sum()

def compute_logp_correct_lin_sub(choices,Nps,Nb,sig_dN,ksig,sig_0,Nmin=1,Nmax=18): #ksig,sig_0,sub
    #sig_Np = ksig*(np.arange(Nmin,Nmax+1)) + sig_0
    sub_NP = np.zeros(4)
    sig_Np = np.concatenate([sub_NP,ksig*(np.arange(5,Nmax+1,1))+sig_0]) # + sig_0
    #sig_Np[0:4] = sub
    pcorrect = compute_pcorrect(choices,Nps,Nb,sig_dN,sig_Np,Nmin)
    log_pcorrect = np.log(pcorrect)
    return log_pcorrect.sum()

def compute_logp_correct_pow(choices,Nps,Nb,sig_dN,ksig,ind,Nmin=1,Nmax=18):
    sig_Np = ksig*np.power((np.arange(Nmin,Nmax+1)),ind)
    pcorrect = compute_pcorrect(choices,Nps,Nb,sig_dN,sig_Np,Nmin)
    log_pcorrect = np.log(pcorrect)
    return log_pcorrect.sum()

def compute_logp_correct_const(choices,Nps,Nb,sig_dN,Nmin=1,Nmax=18):
    sig_Np = np.zeros((Nmax-Nmin+1,))
    pcorrect = compute_pcorrect(choices,Nps,Nb,sig_dN,sig_Np,Nmin)
    log_pcorrect = np.log(pcorrect)
    return log_pcorrect.sum()

def get_pulses(choices,cum_odor,min_pulse,max_pulse):
    all_Nps = np.ceil(cum_odor).astype(np.int)
    ind_selected = np.logical_and(all_Nps>=min_pulse,all_Nps<=max_pulse)
    #print(ind_selected.shape)
    return choices[ind_selected], all_Nps[ind_selected]

def get_odor_profile_actual(odor_profile):
    kernel = np.load('odor_kernel_50ms.npy')
    pulse_onsets = (np.diff(odor_profile)>0).astype(np.float)
    pulse_onsets_shifted = np.concatenate((np.zeros((70,)),pulse_onsets,np.zeros(131)))
    odor_profile_actual = np.convolve(pulse_onsets_shifted,kernel,mode='same')
    return odor_profile_actual

def get_conv_odor(session):
    num_trials = session['num_trials']
    conv_odor = np.zeros((num_trials,5000))
    conv_weights_all = np.zeros((num_trials,5000))
    sniff_kernel = np.load('inhalation_kernel_fine_weights_active.npy')
    sniff_kernel = sniff_kernel/sniff_kernel.mean()
    for i_trial in range(num_trials):
        sniff_raw = np.append(session['trial_pre_breath'][i_trial],session['trial_breath'][i_trial])
        sniff = butter_lowpass_filter(sniff_raw,10,1000,3)
        sniff = (sniff - sniff.mean() +1)/2
        sniff_onset,_ = scipy.signal.find_peaks(sniff,distance=120)
        sniff_onset = sniff_onset[sniff_onset>2250]
        sniff_onset = sniff_onset[sniff_onset<7750]
        sniff_markers = np.zeros((10000,))
        conv_weights = np.zeros((10000,))
        for i in range(len(sniff_onset)-1):
            nsample = sniff_onset[i+1]-sniff_onset[i]
            conv_weights[sniff_onset[i]:sniff_onset[i+1]] = scipy.signal.resample(sniff_kernel,nsample)*250/nsample
        sniff_markers[sniff_onset] = 1
        sniff_sampling_epoch = sniff[2500:7500]
        conv_weights_sampling_epoch = conv_weights[2500:7500]
        odor_command = session['trial_odor'][i_trial]
        odor = get_odor_profile_actual(odor_command)[0:5000]
        odor_effective = odor*conv_weights_sampling_epoch
        conv_odor[i_trial,:] = odor_effective
        conv_weights_all[i_trial,:] = conv_weights_sampling_epoch
    return conv_odor, conv_weights_all

def get_sniff_histogram(session,shuffled=False):
    bins = np.linspace(0,250,15) #16
    num_trials = session['num_trials']
    sniff_hist = np.zeros((num_trials,14)) #15
    sniff_kernel = np.load('inhalation_kernel_fine_weights_active.npy')
    sniff_kernel = sniff_kernel/sniff_kernel.mean()
    
    for i_trial in range(num_trials):
        sniff_raw = np.append(session['trial_pre_breath'][i_trial],session['trial_breath'][i_trial])
        #print(sniff_raw.shape)
        sniff = butter_lowpass_filter(sniff_raw,8,1000,3)
        sniff = (sniff - sniff.mean() +1)/2
        #print(sniff.shape)
        sniff_onset,_ = scipy.signal.find_peaks(sniff,distance=100)
        sniff_onset = sniff_onset[sniff_onset>2250]
        sniff_onset = sniff_onset[sniff_onset<7750]
        sniff_markers = np.zeros((10000,))
        sniff_phase = np.zeros((10000,))

        for i in range(len(sniff_onset)-1):
            nsample = sniff_onset[i+1]-sniff_onset[i]
            sniff_phase[sniff_onset[i]:sniff_onset[i+1]] = scipy.signal.resample(np.arange(0,250),nsample)
        sniff_markers[sniff_onset] = 1
        sniff_sampling_epoch = sniff[2500:7500]
        sniff_phase_sampling_epoch = sniff_phase[2500:]
        odor_command = session['trial_odor'][i_trial]
        odor = get_odor_profile_actual(odor_command)[0:5000]
        if shuffled:
            n_pulses = (np.diff(odor_command)==100).sum()
            valve_onset = np.random.randint(0,5000,(n_pulses,))
        else:
            valve_onset = np.argwhere(np.diff(odor_command)==100)[0:5000]
        odor_onset = valve_onset + 20
        odor_phase = sniff_phase_sampling_epoch[odor_onset].squeeze()
        hist,_ = np.histogram(odor_phase,bins)
        sniff_hist[i_trial,:] = hist
    return sniff_hist

def get_sniff_durations(session):
    num_trials = session['num_trials']
    sniff_durations = np.zeros((num_trials,50))
    for i_trial in range(num_trials):
        sniff_raw = np.append(session['trial_pre_breath'][i_trial],session['trial_breath'][i_trial])
        sniff = butter_lowpass_filter(sniff_raw,8,1000,3)
        sniff = (sniff - sniff.mean() +1)/2
        sniff_onset,_ = scipy.signal.find_peaks(-sniff,distance=100)
        sniff_onset = sniff_onset[sniff_onset>2250]
        sniff_onset = sniff_onset[sniff_onset<7750]
        sniff_markers = np.zeros((10000,))
        sniff_duration = np.diff(sniff_onset)
        sniff_durations[i_trial,:len(sniff_duration)] = sniff_duration
    return sniff_durations

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scipy.signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y

def get_autocorr(session,n):
    high_trials = session['high_trials']
    autocorr = list()
    for i in range(n):
        autocorr.append(np.corrcoef(high_trials[i+1:],high_trials[:-(i+1)])[0][1])
    return np.array(autocorr)

def get_colors(inp, colormap, vmin=None, vmax=None):
    norm = plt.Normalize(vmin, vmax)
    return colormap(norm(inp))

In [3]:
path = os.getcwd() + '/Session/'
savepath = os.getcwd() + '/Figures/test/'
savepath_pdf = os.getcwd() + '/Figures/test/pdf/'
savepath_png = os.getcwd() + '/Figures/test/png/'
savepath_svg = os.getcwd() + '/Figures/test/svg/'


dir_list = os.listdir(path)
np_savepath = os.getcwd() + '/MLE_arrays/'

print(os.getcwd())

/Users/boero/data_oea_project


In [4]:
# Compute psychometrics of each animal
num_bins = 20
half_num_bins = 8
min_pulse = 1
max_pulse = 20
num_hist = 6
pulse_bins = np.linspace(min_pulse,max_pulse,max_pulse-min_pulse+1)

phigh_list = list()
performance_list = list()
bin_counts_list = list()
phigh_hist_list = list()
performance_hist_list = list()
bin_counts_hist_list = list()
all_high_choices_list = list()
all_cum_odor_list = list()
all_cum_conv_odor_list = list()
all_correct_trials_list = list()
all_sniff_durations_list = list()
all_sniff_hist_list = list()
all_sniff_hist_shuffled_list = list()

all_high_choices_hist_list = list()
all_cum_odor_hist_list = list()
all_correct_trials_hist_list = list()
all_sniff_durations_hist_list = list()
all_sniff_hist_h_list = list()
all_sniff_hist_shuffled_h_list = list()
all_previous_trial_list = list()
autocorr_list = list()

animal_list = [['Tabby'],
               ['Bengal'],
               ['Shorthair'],
               ['Banner'],
               ['Natasha'],
               ['Stark'],
               ['Peterparker']]

for animals in animal_list:
    fname_list = list()

    for date_number in range(20190419,20220526):
        for animal in animals:
            for session_id in range(0,10):
                session_string = str(session_id)
                date_string = str(date_number)
                fname = animal + '_' + date_string + '_' + session_string + '.pickle'
                cell_fname = path + 'session_' + fname
                if os.path.isfile(cell_fname):
                    fname_list.append(fname)

    num_all_trials = 0
    
    all_trial_odor = np.zeros((0,5000))
    all_conv_weights = np.zeros((0,5000))
    all_conv_odor = np.zeros((0,5000))
    all_cum_odor = np.zeros((0,))
    all_cum_conv_odor = np.zeros((0,))
    all_high_trials = np.zeros((0,))
    all_high_choices = np.zeros((0,))
    all_correct_trials = np.zeros((0,))
    all_sniff_durations = np.zeros(0,)
    all_sniff_hist = np.zeros((0,14)) #15
    all_sniff_hist_shuffled = np.zeros((0,14)) #15
    
    all_trial_odor_hist = np.zeros((0,5000))
    all_cum_odor_hist = np.zeros((0,))
    all_high_trials_hist = np.zeros((0,))
    all_high_choices_hist = np.zeros((0,))
    all_correct_trials_hist = np.zeros((0,))
    all_sniff_durations_hist = np.zeros(0,)
    all_sniff_hist_h = np.zeros((0,14)) #15
    all_sniff_hist_shuffled_h = np.zeros((0,14)) #15
    all_previous_trials = np.zeros((0,num_hist))
    
    for i_file in tnrange(len(fname_list)):
        session_id = 0
        fname = fname_list[i_file]
        session_fname = path + 'session_' + fname
        if os.path.isfile(session_fname):
            with open(session_fname,'rb') as handle:
                session_list = pickle.load(handle)
                session = session_list[0]
                if session_list[0]['type'] == 'random':
                    if session_list[0]['delay_time'] == 5:
                        #if round(session_list[0]['high_count']/session_list[0]['low_count']) == 3:
                            #if session_list[0]['low_count'] < 5:

                        num_trials = session['num_trials']
                        print(session_fname, session_list[0]['high_count'], session_list[0]['low_count'])
                        autocorr_list.append(get_autocorr(session,num_hist))
                        session['conv_trial_odor'],session['conv_weights'] = get_conv_odor(session)
                        session['sniff_durations'] = get_sniff_durations(session)
                        non_idle_trials = np.invert(session['idle_trials'])
                        correct_trials = session['correct_trials'][non_idle_trials]
                        high_trials = session['high_trials'][non_idle_trials]
                        low_trials = session['low_trials'][non_idle_trials]
                        high_choices = (correct_trials==high_trials)
                        trial_odor = session['trial_odor'][non_idle_trials]
                        sniff_durations = session['sniff_durations'][non_idle_trials]
                        sniff_durations = sniff_durations[sniff_durations>0] 
                        #pasting all the sniff durations of the session in just one array
                        sniff_hist = get_sniff_histogram(session,False)[non_idle_trials]
                        sniff_hist_shuffled = get_sniff_histogram(session,True)[non_idle_trials]
                        cum_odor = trial_odor.sum(axis=1)/5000
                        num_trials_taken = len(high_trials) - num_hist
                        previous_ind = np.arange(num_trials_taken).reshape(-1,1)+np.arange(0,num_hist)
                        previous_trials = high_choices[previous_ind]

                        all_trial_odor = np.append(all_trial_odor, trial_odor[:,:5000], axis=0)
                        all_trial_odor_hist = np.append(all_trial_odor_hist, trial_odor[num_hist:,:5000], axis=0)
                        all_correct_trials = np.append(all_correct_trials,correct_trials)
                        all_correct_trials_hist = np.append(all_correct_trials_hist,correct_trials[num_hist:])
                        all_high_trials = np.append(all_high_trials,high_trials)
                        all_high_trials_hist = np.append(all_high_trials_hist,high_trials[num_hist:])
                        all_high_choices = np.append(all_high_choices,high_choices)
                        all_high_choices_hist = np.append(all_high_choices_hist,high_choices[num_hist:])
                        all_cum_odor = np.append(all_cum_odor,cum_odor)
                        all_cum_odor_hist = np.append(all_cum_odor_hist,cum_odor[num_hist:])
                        all_sniff_durations = np.append(all_sniff_durations,sniff_durations)
                        all_sniff_durations_hist = np.append(all_sniff_durations_hist,sniff_durations[num_hist:])
                        all_sniff_hist = np.append(all_sniff_hist,sniff_hist,axis=0)
                        all_sniff_hist_h = np.append(all_sniff_hist_h,sniff_hist[num_hist:],axis=0)
                        all_sniff_hist_shuffled = np.append(all_sniff_hist_shuffled,sniff_hist_shuffled,axis=0)
                        all_sniff_hist_shuffled_h = np.append(all_sniff_hist_shuffled_h,sniff_hist_shuffled[num_hist:],axis=0)
                        all_previous_trials = np.append(all_previous_trials,previous_trials,axis=0)
                        #print(len(all_cum_odor))
                        #print(len(all_cum_odor_hist))

        high_seven = all_cum_odor>8
        low_eight = all_cum_odor<=8
        high_seven_hist = all_cum_odor_hist>8
        low_eight_hist = all_cum_odor_hist<=8
        all_low_trials = np.invert(all_high_trials.astype(np.bool))
        all_low_trials_hist = np.invert(all_high_trials_hist.astype(np.bool))
        bad_trials_1 = np.logical_and(high_seven,all_low_trials)
        bad_trials_2 = np.logical_and(low_eight,all_high_trials)
        bad_trials = np.logical_or(bad_trials_1,bad_trials_2)
        good_trials = np.invert(bad_trials)
        bad_trials_hist_1 = np.logical_and(high_seven_hist,all_low_trials_hist)
        bad_trials_hist_2 = np.logical_and(low_eight_hist,all_high_trials_hist)
        bad_trials_hist = np.logical_or(bad_trials_hist_1,bad_trials_hist_2)
        good_trials_hist = np.invert(bad_trials_hist)
        #print(len(good_trials))
        #print(len(good_trials_hist))
        
        all_correct_trials = all_correct_trials[good_trials]
        all_correct_trials_hist = all_correct_trials_hist[good_trials_hist]
        all_high_trials = all_high_trials[good_trials]
        all_high_trials_hist = all_high_trials_hist[good_trials_hist]
        all_low_trials = all_low_trials[good_trials]
        all_low_trials_hist = all_low_trials_hist[good_trials_hist]
        all_high_choices = all_high_choices[good_trials]
        all_cum_odor = all_cum_odor[good_trials]
        all_trial_odor = all_trial_odor[good_trials,:]
        all_sniff_hist = all_sniff_hist[good_trials,:]
        all_sniff_hist_shuffled = all_sniff_hist_shuffled[good_trials,:]
        all_high_choices_hist = all_high_choices_hist[good_trials_hist]
        all_cum_odor_hist = all_cum_odor_hist[good_trials_hist]
        all_trial_odor_hist = all_trial_odor_hist[good_trials_hist,:]
        all_sniff_hist_h = all_sniff_hist_h[good_trials_hist,:]
        all_sniff_hist_shuffled_h = all_sniff_hist_shuffled_h[good_trials_hist,:]
        all_previous_trials = all_previous_trials[good_trials_hist,:]
    
    #np.save('sniff_analysis_correct/' + animals[0] + '_' + 'conv_odor.npy', all_conv_odor)
#     np.save('sniff_analysis_correct/' + animals[0] + '_' + 'all_high_trials.npy', all_high_trials_hist)
#     np.save('sniff_analysis_correct/' + animals[0] + '_' + 'all_high_choices.npy', all_high_choices_hist)
#     np.save('sniff_analysis_correct/' + animals[0] + '_' + 'all_true_pulses_number.npy', all_cum_odor_hist)
#     np.save('sniff_analysis_correct/' + animals[0] + '_' + 'all_odor_commands.npy', all_trial_odor_hist)
#     #np.save('sniff_analysis_correct/' + animals[0] + '_' + 'all_conv_weights.npy', all_conv_weights)
    all_correct_trials_list.append(all_correct_trials)
    all_high_choices_list.append(all_high_choices)
    all_cum_odor_list.append(all_cum_odor)
    #all_cum_conv_odor_list.append(all_cum_conv_odor)
    all_sniff_durations_list.append(all_sniff_durations)
    all_sniff_hist_list.append(all_sniff_hist)
    all_sniff_hist_shuffled_list.append(all_sniff_hist_shuffled)
    all_previous_trial_list.append(all_previous_trials)
    
    all_correct_trials_hist_list.append(all_correct_trials_hist)
    all_high_choices_hist_list.append(all_high_choices_hist)
    all_cum_odor_hist_list.append(all_cum_odor_hist)
    #all_cum_conv_odor_list.append(all_cum_conv_odor)
    all_sniff_durations_hist_list.append(all_sniff_durations_hist)
    all_sniff_hist_h_list.append(all_sniff_hist_h)
    all_sniff_hist_shuffled_h_list.append(all_sniff_hist_shuffled_h)    
    phigh = np.zeros((num_bins,2))
    performance = np.zeros((num_bins,2))
    phigh_hist = np.zeros((num_bins,2))
    performance_hist = np.zeros((num_bins,2))
    bin_counts = np.zeros((num_bins,))
    bin_counts_h = np.zeros((num_bins,))

    pulse_bin_ind = np.ceil(all_cum_odor).astype(np.int)
    pulse_bin_ind_h = np.ceil(all_cum_odor_hist).astype(np.int)
 
    for i in range(num_bins):
        selected = all_high_choices[pulse_bin_ind==pulse_bins[i]]
        selected_hist = all_high_choices_hist[pulse_bin_ind_h==pulse_bins[i]]
        selected_correct = all_correct_trials[pulse_bin_ind==pulse_bins[i]]
        selected_correct_hist = all_correct_trials_hist[pulse_bin_ind_h==pulse_bins[i]]
        num_selected = len(selected)
        num_selected_h = len(selected_hist)
        bin_counts[i] = num_selected
        bin_counts_h[i] = num_selected_h
        phigh[i,0] = selected.mean()
        phigh_hist[i,0] = selected_hist.mean()
        phigh[i,1] = 1.69*selected.std()/np.sqrt(num_selected)
        phigh_hist[i,1] = 1.69*selected.std()/np.sqrt(num_selected_h)
        performance[i,0] = selected_correct.mean()
        performance[i,1] = 1.69*selected_correct.std()/np.sqrt(num_selected)
        performance_hist[i,0] = selected_correct_hist.mean()
        performance_hist[i,1] = 1.69*selected_correct_hist.std()/np.sqrt(num_selected_h)
    phigh_list.append(phigh)
    performance_list.append(performance)
    bin_counts_list.append(bin_counts)
    phigh_hist_list.append(phigh_hist)
    performance_hist_list.append(performance_hist)
    bin_counts_hist_list.append(bin_counts_h)

autocorr_mat = np.array(autocorr_list)

  0%|          | 0/18 [00:00<?, ?it/s]

/Users/boero/data_oea_project/Session/session_Tabby_20190419_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190423_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190426_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190427_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190430_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190501_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190503_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190504_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190505_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190510_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Tabby_20190511_0.pickle 

  0%|          | 0/22 [00:00<?, ?it/s]

/Users/boero/data_oea_project/Session/session_Bengal_20190419_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190422_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190423_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190426_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190427_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190430_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190501_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190502_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190503_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_20190504_0.pickle 12.5 4.166666666666667
/Users/boero/data_oea_project/Session/session_Bengal_2019050

  0%|          | 0/17 [00:00<?, ?it/s]

...

/Users/boero/data_oea_project/Session/session_Natasha_20220322_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Natasha_20220323_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Natasha_20220324_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Natasha_20220325_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Natasha_20220326_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Natasha_20220328_0.pickle 11.037527593818984 5.512679162072767
/Users/boero/data_oea_project/Session/session_Natasha_20220330_0.pickle 11.037527593818984 5.518763796909492
/Users/boero/data_oea_project/Session/session_Natasha_20220418_1.pickle 12.594458438287154 2.0990764063811924
/Users/boero/data_oea_project/Session/session_Natasha_20220418_2.pickle 12.594458438287154 2.0990764063811924
/Users/boero/data

  0%|          | 0/154 [00:00<?, ?it/s]

/Users/boero/data_oea_project/Session/session_Stark_20220322_2.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220322_3.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220323_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220323_1.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220324_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220324_1.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220325_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220325_2.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Stark_20220326_1.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session

  0%|          | 0/165 [00:00<?, ?it/s]

/Users/boero/data_oea_project/Session/session_Peterparker_20220330_0.pickle 15.384615384615385 5.128205128205129
/Users/boero/data_oea_project/Session/session_Peterparker_20220401_0.pickle 12.0 4.0
/Users/boero/data_oea_project/Session/session_Peterparker_20220404_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Peterparker_20220404_2.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Peterparker_20220405_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Peterparker_20220406_1.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Peterparker_20220407_0.pickle 12.31527093596059 4.1017227235438884
/Users/boero/data_oea_project/Session/session_Peterparker_20220408_0.pickle 11.037527593818984 5.518763796909492
/Users/boero/data_oea_project/Session/session_Peterparker_20220410_0.pickle 11.037527593818984 5.518763796909492
/Users/boer